In [2]:
import anthropic
import base64
import os
import pandas as pd
import io
import time
import re
import json

import config

## Step one: Initialize Anthropic API client

In [2]:
# set anthropic API key
api_key = config.anthropic_api_key

In [3]:
client = anthropic.Anthropic(api_key = api_key)

## Step two: Compose prompt segments + define prompt function

Below, we define a prompt function that will request a response from the Claude genAI model via the Anthropic API. Keep in mind the following three pieces of the prompt:

* The user message
* The system message
* The assistant prefix

More information via the [Anthropic API technical documentation](https://docs.anthropic.com/en/api/messages)

Note that in order to accomodate different possible `value` columns, we are using a function parameter to define the user message based on the year.

TO ADD: Additional example, but this time with the full problem (the country names + values just like any image we'll ask later). This may improve quality from the beginning.

In [4]:
assistant_prefix = '''[{"DESCRIPCION":'''

In [31]:
# take in an image (e.g. from a PDF page); query claude API; return table
def compose_image_message(image_path, year, type_of_data = "import", assistant_pre = ""):
    # read image
    with open(image_path, "rb") as f:
        image_data = f.read()
        base64_image = base64.b64encode(image_data).decode()

    with open("tall_complete_numeric_training_data.jpg", "rb") as f:
        training_image_data = f.read()
        training_base64_image = base64.b64encode(training_image_data).decode()

    with open("full_page_test.jpg", "rb") as f:
        full_table_training_image_data = f.read()
        full_table_training_base64_image = base64.b64encode(training_image_data).decode()

    # set our three prompt components:
    # first, set a value column to use based on the year
    if year < 1964:
        value_type = "DOLARES"
    elif year == 1964:
        value_type = "PESOS CA"
    else:
    	value_type = "PESOS CENTROAMERICANOS"

    # now define our three components

    pre_font_training_prompt = '''I am going to send you an example of the typewriter font you will see in the main image in this prompt. Please look at the following training image and learn the shapes of every digit from 0 to 9. This is the values you should see, in order:
   138085
   94128
   7404
   2215
   1659
   1151
   14
   3
   47268
   47268
   17058
   19529
   3146

   398476
   230838
   245119
   53724
   14713
   39907
   23658
   2421
   3586
   21933
   14146
   1079
   10857
   1184
   12442
   137
   1662
   1964
   1250
   168
   670
   285

   331441985
   59083255
   17813355
   9032695
   7150080
   5627895
   3490585
   2328165
   2056845
   1923180
   718200
   222775
   133000
   67830
   63840
   33915
   6650
  
  
  '''

    pre_complete_task_training_prompt = '''I am now going to send you an example of the historical table reading task, so you learn how to detect columns, rows, and values correctly. Let's say I am asking you to create a list of dictionaries corresponding to rows in this table. Create a JSON list that follows the historical trade table and includes one dictionary per row in the image. The output should include the following three keys: DESCRIPCION, PAIS, and VALOR.  Please format each value in the row as follows. For the DESCRIPCION value, include only the string of numbers and spaces that appear in the DESCRIPCION table column - this value looks something like 051 07 02 00. Treat this data like a string and include leading zeros and spaces. Null/blank values are acceptable - return them as empty strings. For the PAIS value, please return exactly the PAIS value (examples: HONDURAS, U A E). Double check that you have the PAIS string exactly right - double check the letters and respect the sequence. Include every single row that contains a PAIS value. For VALOR, please return the value in the PESOS CA column. Here is the output you should achieve:

[{'DESCRIPCION': '', 'PAIS': 'ITALIA', 'VALOR': 67732},

 {'DESCRIPCION': '', 'PAIS': 'ALEMANOC', 'VALOR': 24607},

 {'DESCRIPCION': '', 'PAIS': 'ELSALVAD', 'VALOR': 1045},

 {'DESCRIPCION': '', 'PAIS': 'E U A', 'VALOR': 716},

 {'DESCRIPCION': '', 'PAIS': 'AUSTRIA', 'VALOR': 524},

 {'DESCRIPCION': '', 'PAIS': 'REINOUNI', 'VALOR': 483},

 {'DESCRIPCION': '732 03 01', 'PAIS': 'E U A', 'VALOR': 586380},

 {'DESCRIPCION': '732 03 01', 'PAIS': 'ALEMANOC', 'VALOR': 95618},

 {'DESCRIPCION': '732 03 01', 'PAIS': 'REINOUNI', 'VALOR': 39469},

 {'DESCRIPCION': '732 03 01', 'PAIS': 'JAPON', 'VALOR': 9450},

 {'DESCRIPCION': '732 03 01', 'PAIS': 'ITALIA', 'VALOR': 1650},

 {'DESCRIPCION': '732 03 01', 'PAIS': 'AUSTRIA', 'VALOR': 1648},

 {'DESCRIPCION': '732 03 01', 'PAIS': 'FRANCIA', 'VALOR': 1512},

 {'DESCRIPCION': '732 03 01', 'PAIS': 'ELSALVAD', 'VALOR': 1100},

 {'DESCRIPCION': '732 03 01', 'PAIS': 'PANAMA', 'VALOR': 800},

 {'DESCRIPCION': '732 03 01', 'PAIS': 'NICARAGU', 'VALOR': 750},

 {'DESCRIPCION': '732 03 02', 'PAIS': 'E U A', 'VALOR': 863996},

 {'DESCRIPCION': '732 03 02', 'PAIS': 'JAPON', 'VALOR': 252324},

 {'DESCRIPCION': '732 03 02', 'PAIS': 'REINOUNI', 'VALOR': 197340},

 {'DESCRIPCION': '732 03 02', 'PAIS': 'ALEMANOC', 'VALOR': 194809},

 {'DESCRIPCION': '732 03 02', 'PAIS': 'FRANCIA', 'VALOR': 15779},

 {'DESCRIPCION': '732 03 02', 'PAIS': 'ITALIA', 'VALOR': 11449},

 {'DESCRIPCION': '732 03 02', 'PAIS': 'AUSTRIA', 'VALOR': 3494},

 {'DESCRIPCION': '732 03 02', 'PAIS': 'PANAMA', 'VALOR': 725},

 {'DESCRIPCION': '732 03 02', 'PAIS': 'MARRUECO', 'VALOR': 600},

 {'DESCRIPCION': '732 03 03', 'PAIS': 'E U A', 'VALOR': 98120},

 {'DESCRIPCION': '732 03 03', 'PAIS': 'JAPON', 'VALOR': 8920},

 {'DESCRIPCION': '732 03 03', 'PAIS': 'REINOUNI', 'VALOR': 961},

 {'DESCRIPCION': '732 04 00', 'PAIS': 'ALEMANOC', 'VALOR': 10204},

 {'DESCRIPCION': '732 04 00', 'PAIS': 'E U A', 'VALOR': 1100}]
        
    '''

    pre_image_prompt = "Ok. Now that you recognize the typewriter font. Read the following image. This is a historical trade table. Take extra care to identify the correct numeric values and digits - this is a badly scanned image and you must double check any image values."
    
    user_message = "Create a JSON list that follows the historical trade table and includes one dictionary per row in the image. The output should include the following three keys: DESCRIPCION, PAIS, and VALOR.  Please format each value in the row as follows. For the DESCRIPCION value, include only the string of numbers and spaces that appear in the DESCRIPCION table column - this value looks something like 051 07 02 00. Treat this data like a string and include leading zeros and spaces. Null/blank values are acceptable - return them as empty strings. For the PAIS value, please return exactly the PAIS value (examples: HONDURAS, U A E). Double check that you have the PAIS string exactly right - double check the letters and respect the sequence. Include every single row that contains a PAIS value. For VALOR, please return the value in the " + value_type + " column. Please double-check your accuracy scanning this number."

    # now, compose our message

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": pre_font_training_prompt


                },
                                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": training_base64_image
                    }
                },
                {
                    "type": "text",
                    "text": pre_complete_task_training_prompt


                },
                                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": full_table_training_base64_image
                    }
                },
                                {
                    "type": "text",
                    "text": pre_image_prompt


                },
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": base64_image
                    }
                },
                 {
                    "type": "text",
                    "text":  user_message


                }
                
            ]
        },
        {
            "role": "assistant",
            "content": assistant_pre }
    ]

    return messages

In [6]:
# set a system prompt for claude
system_prompt = "You are a historian of economics data, looking to create detailed, error-free JSON list data based on scanned typerwriter tables. Only include the JSON list in your result. Return the complete results, and make sure to properly close the JSON list in your response."

In [7]:
def submit_image_prompt(message, api_client, system, assistant_pre):
    # send message

    response = api_client.messages.create(
        model="claude-3-5-sonnet-20241022",
        messages=message,
        system=system,
        max_tokens=8192
    )


    # return just the text response
    text_content = response.content[0].text


    # we must manually add the JSON prefix text back
    text_content_final = assistant_pre + text_content
    

    # now return the output
    text_content_final

    return text_content_final


Note that for this project, we need to conditionally modify the message for a key column (`values`) based on the `year` value in the data.

## Step three: Test the prompt and parse the result as JSON

In [8]:
# test

# test with existing image
start_time = time.time()
test_message = compose_image_message("images_import_only/file1963_page95.jpg", year = 1963, type_of_data = "import", assistant_pre = assistant_prefix)
test_output = submit_image_prompt(test_message, client, system_prompt, assistant_prefix)
print(f"Time taken: {time.time() - start_time} seconds")

Time taken: 18.538599967956543 seconds


In [9]:
test_output

'[{"DESCRIPCION":  "552 02 01", "PAIS": "ELSALVAD", "VALOR": 54272},\n{"DESCRIPCION":  "552 02 01", "PAIS": "ESPANA", "VALOR": 18376},\n{"DESCRIPCION":  "552 02 01", "PAIS": "E U A", "VALOR": 10648},\n{"DESCRIPCION":  "552 02 01", "PAIS": "REINOUNI", "VALOR": 9814},\n{"DESCRIPCION":  "552 02 01", "PAIS": "FRANCIA", "VALOR": 7425},\n{"DESCRIPCION":  "552 02 01", "PAIS": "ALEMANOC", "VALOR": 2478},\n{"DESCRIPCION":  "552 02 01", "PAIS": "SUIZA", "VALOR": 1188},\n{"DESCRIPCION":  "552 02 01", "PAIS": "MEXICO", "VALOR": 312},\n{"DESCRIPCION":  "552 02 01", "PAIS": "HOLANDA", "VALOR": 326},\n{"DESCRIPCION":  "552 02 01", "PAIS": "BAHAM", "VALOR": 59},\n{"DESCRIPCION":  "552 02 01", "PAIS": "PORTUGAL", "VALOR": 41},\n{"DESCRIPCION":  "552 02 01", "PAIS": "PANAMA", "VALOR": 35},\n{"DESCRIPCION":  "552 02 01", "PAIS": "ITALIA", "VALOR": 19},\n{"DESCRIPCION":  "552 02 01", "PAIS": "COLOMBIA", "VALOR": 4},\n{"DESCRIPCION":  "552 02 01", "PAIS": "ARUBA", "VALOR": 2},\n{"DESCRIPCION":  "552 02 02"

In [10]:
# test the output - does it behave like JSON?
json.loads(test_output)

[{'DESCRIPCION': '552 02 01', 'PAIS': 'ELSALVAD', 'VALOR': 54272},
 {'DESCRIPCION': '552 02 01', 'PAIS': 'ESPANA', 'VALOR': 18376},
 {'DESCRIPCION': '552 02 01', 'PAIS': 'E U A', 'VALOR': 10648},
 {'DESCRIPCION': '552 02 01', 'PAIS': 'REINOUNI', 'VALOR': 9814},
 {'DESCRIPCION': '552 02 01', 'PAIS': 'FRANCIA', 'VALOR': 7425},
 {'DESCRIPCION': '552 02 01', 'PAIS': 'ALEMANOC', 'VALOR': 2478},
 {'DESCRIPCION': '552 02 01', 'PAIS': 'SUIZA', 'VALOR': 1188},
 {'DESCRIPCION': '552 02 01', 'PAIS': 'MEXICO', 'VALOR': 312},
 {'DESCRIPCION': '552 02 01', 'PAIS': 'HOLANDA', 'VALOR': 326},
 {'DESCRIPCION': '552 02 01', 'PAIS': 'BAHAM', 'VALOR': 59},
 {'DESCRIPCION': '552 02 01', 'PAIS': 'PORTUGAL', 'VALOR': 41},
 {'DESCRIPCION': '552 02 01', 'PAIS': 'PANAMA', 'VALOR': 35},
 {'DESCRIPCION': '552 02 01', 'PAIS': 'ITALIA', 'VALOR': 19},
 {'DESCRIPCION': '552 02 01', 'PAIS': 'COLOMBIA', 'VALOR': 4},
 {'DESCRIPCION': '552 02 01', 'PAIS': 'ARUBA', 'VALOR': 2},
 {'DESCRIPCION': '552 02 02', 'PAIS': 'ELSALV

In [11]:
# Try to convert / regularize as a dataframe table
test_output_as_pd = pd.DataFrame(json.loads(test_output))

In [12]:
test_output_as_pd

,DESCRIPCION,PAIS,VALOR
0,552 02 01,ELSALVAD,54272
1,552 02 01,ESPANA,18376
2,552 02 01,E U A,10648
3,552 02 01,REINOUNI,9814
4,552 02 01,FRANCIA,7425
5,552 02 01,ALEMANOC,2478
6,552 02 01,SUIZA,1188
7,552 02 01,MEXICO,312
8,552 02 01,HOLANDA,326
9,552 02 01,BAHAM,59


## Step four: Run workflow on complete dataset

In [14]:
# create metadata based on import/export images in folders
# this metadata will drive the querying process next

all_import_images = os.listdir('images_import_only')
all_export_images =  os.listdir('images_export_only')

all_image_metadata = []

In [15]:
for image_filename in all_import_images:
    try:
        year = int(image_filename[4:8])
        image_page = int(image_filename[13:-4])
        current_image = {'year': year
                         ,'page': image_page
                         ,'source_type': 'import'
                         ,'filename': image_filename
                         ,'full_path': 'images_import_only/' + image_filename}
        all_image_metadata.append(current_image)
    except:
        pass

for image_filename in all_export_images:
    try:
        year = int(image_filename[4:8])
        image_page = int(image_filename[13:-4])
        current_image = {'year': year
                         ,'page': image_page
                         ,'source_type': 'export'
                         ,'filename': image_filename
                         ,'full_path': 'images_export_only/' + image_filename}
        all_image_metadata.append(current_image)
    except:
        pass

                

In [16]:
all_images_df = pd.DataFrame(all_image_metadata).sort_values(['source_type', 'year', 'page'], ascending=[False, True, True])
all_images_df

,year,page,source_type,filename,full_path
3143,1960,26,import,file1960_page26.jpg,images_import_only/file1960_page26.jpg
2998,1960,27,import,file1960_page27.jpg,images_import_only/file1960_page27.jpg
34,1960,28,import,file1960_page28.jpg,images_import_only/file1960_page28.jpg
182,1960,29,import,file1960_page29.jpg,images_import_only/file1960_page29.jpg
3217,1960,30,import,file1960_page30.jpg,images_import_only/file1960_page30.jpg
...,...,...,...,...,...
3874,1982,349,export,file1982_page349.jpg,images_export_only/file1982_page349.jpg
4286,1982,350,export,file1982_page350.jpg,images_export_only/file1982_page350.jpg
4346,1982,351,export,file1982_page351.jpg,images_export_only/file1982_page351.jpg
4445,1982,352,export,file1982_page352.jpg,images_export_only/file1982_page352.jpg


In [36]:
just_type = all_images_df[all_images_df["source_type"] == "export"]
just_type_and_year = just_type[(just_type["year"] == 1964)  ]
target_items_as_dict = just_type_and_year.to_dict(orient="records")
target_items_as_dict

[{'year': 1964,
  'page': 371,
  'source_type': 'export',
  'filename': 'file1964_page371.jpg',
  'full_path': 'images_export_only/file1964_page371.jpg'},
 {'year': 1964,
  'page': 372,
  'source_type': 'export',
  'filename': 'file1964_page372.jpg',
  'full_path': 'images_export_only/file1964_page372.jpg'},
 {'year': 1964,
  'page': 373,
  'source_type': 'export',
  'filename': 'file1964_page373.jpg',
  'full_path': 'images_export_only/file1964_page373.jpg'},
 {'year': 1964,
  'page': 374,
  'source_type': 'export',
  'filename': 'file1964_page374.jpg',
  'full_path': 'images_export_only/file1964_page374.jpg'},
 {'year': 1964,
  'page': 375,
  'source_type': 'export',
  'filename': 'file1964_page375.jpg',
  'full_path': 'images_export_only/file1964_page375.jpg'},
 {'year': 1964,
  'page': 376,
  'source_type': 'export',
  'filename': 'file1964_page376.jpg',
  'full_path': 'images_export_only/file1964_page376.jpg'},
 {'year': 1964,
  'page': 377,
  'source_type': 'export',
  'filename'

In [37]:
# ALL imports LET US TRY
type_for_this_run = "export"

for image in target_items_as_dict:
    image_full_path = image["full_path"]
    image_filename = image["filename"]
    image_year = int(image["year"])
    image_page = image["page"]
    start_time = time.time()
    test_message = compose_image_message(image_full_path, year = image_year, type_of_data = type_for_this_run, assistant_pre = assistant_prefix)
    test_output = submit_image_prompt(test_message, client, system_prompt, assistant_prefix)
    print(f"Time taken: {time.time() - start_time} seconds")
    # try to parse the JSON and conver to csv. if it works, save as CSV. if not, save as raw TXT
    try:
        parsed_json = json.loads(test_output)
        final_dataframe = pd.DataFrame(parsed_json)
        output_filename = image_filename[0:-4]
        output_filename = output_filename + "_" + type_for_this_run + ".csv"
        final_dataframe["year"] = image_year
        final_dataframe["pdf_image_page"] = image_page
        final_dataframe.to_csv("data_" + type_for_this_run + "_only/" + output_filename, index=False)
        print("success for " + image_filename + " response")
    except:
        print("parsing error! will save as text.")
        output_filename = image_filename[0:-4]
        output_filename = output_filename + "_export_parsefail.txt"
        with open("data_export_only/" + output_filename, "w") as text_file:
            text_file.write(test_output)
        print("written as raw text.")

    time.sleep(5)


Time taken: 11.767333984375 seconds
success for file1964_page371.jpg response
Time taken: 8.985253810882568 seconds
success for file1964_page372.jpg response
Time taken: 12.036375045776367 seconds
success for file1964_page373.jpg response
Time taken: 9.54576301574707 seconds
success for file1964_page374.jpg response
Time taken: 7.525031805038452 seconds
success for file1964_page375.jpg response
Time taken: 5.570838928222656 seconds
success for file1964_page376.jpg response
Time taken: 8.468228101730347 seconds
success for file1964_page377.jpg response
Time taken: 8.737091064453125 seconds
success for file1964_page378.jpg response
Time taken: 7.084295988082886 seconds
success for file1964_page379.jpg response
Time taken: 9.610235214233398 seconds
success for file1964_page380.jpg response
Time taken: 7.412890911102295 seconds
success for file1964_page381.jpg response
Time taken: 10.668667078018188 seconds
success for file1964_page382.jpg response
Time taken: 7.1312620639801025 seconds
su

In [ ]:
# add slight tweaked version for imports here!!!

In [21]:
def concatenate_csvs(folder_path, output_filename="combined.csv"):
    """
    Concatenates all CSV files in a folder into a single CSV file.

    Args:
        folder_path (str): The path to the folder containing the CSV files.
        output_filename (str, optional): The name of the output CSV file. Defaults to "combined.csv".
    """
    all_files = os.listdir(folder_path)
    all_files.sort()
    csv_files = [f for f in all_files if f.endswith('.csv')]
    
    if not csv_files:
        print("No CSV files found in the specified folder.")
        return

    all_df = []
    print(all_df)
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        all_df.append(df)

    combined_df = pd.concat(all_df, ignore_index=True)
    output_path = os.path.join(folder_path, output_filename)
    combined_df.to_csv(output_path, index=False)
    print(f"Successfully concatenated {len(csv_files)} CSV files to '{output_path}'")

In [23]:
# Example usage:
folder_path = "/Users/casswilkinsonsaldana/Documents/GitHub/dataservices_cr_genAI_ocr/data_import_only/"  # Replace with the actual path to your folder
concatenate_csvs(folder_path, output_filename = "combined_imports_1964.csv")

[]
Successfully concatenated 385 CSV files to '/Users/casswilkinsonsaldana/Documents/GitHub/dataservices_cr_genAI_ocr/data_import_only/combined_imports_1964.csv'


In [24]:
# Example usage:
folder_path = "/Users/casswilkinsonsaldana/Documents/GitHub/dataservices_cr_genAI_ocr/data_export_only/"  # Replace with the actual path to your folder
concatenate_csvs(folder_path, output_filename = "combined_exports_1964.csv")

[]
Successfully concatenated 81 CSV files to '/Users/casswilkinsonsaldana/Documents/GitHub/dataservices_cr_genAI_ocr/data_export_only/combined_exports_1964.csv'
